In [2]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC
from sklearn.svm import LinearSVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn import linear_model
from sklearn.naive_bayes import BernoulliNB
from sklearn.naive_bayes import MultinomialNB
from sklearn.preprocessing import scale

# sklearn metrics
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
from sklearn.metrics import roc_curve 
from sklearn.metrics import auc
from sklearn.metrics import classification_report
from sklearn.learning_curve import learning_curve
from sklearn.cross_validation import cross_val_score
from sklearn.cross_validation import train_test_split


# images
from scipy.ndimage import convolve
from skimage.feature import hog
from skimage import draw, data, io, segmentation, color, exposure
from skimage.measure import regionprops
from skimage.filters import threshold_otsu
from skimage.transform import resize 
from skimage.transform import warp 
from PIL import Image

# basics
import pickle
import pandas as pd
import numpy as np
from pprint import pprint
from PIL import Image
from pytesseract import image_to_string



ImportError: No module named 'pytesseract'

In [3]:
!pip3 install pytesseract

Cleaning up...


  Using cached tesseract-ocr-0.0.1.tar.gz
  Running setup.py bdist_wheel for tesseract-ocr ... - \ error
  Complete output from command /home/it/anaconda3/bin/python -u -c "import setuptools, tokenize;__file__='/tmp/pip-build-6fn8dbad/tesseract-ocr/setup.py';f=getattr(tokenize, 'open', open)(__file__);code=f.read().replace('\r\n', '\n');f.close();exec(compile(code, __file__, 'exec'))" bdist_wheel -d /tmp/tmpb2kkpo11pip-wheel- --python-tag cp35:
  running bdist_wheel
  running build
  running build_py
  file tesseract_ocr.py (for module tesseract_ocr) not found
  file tesseract_ocr.py (for module tesseract_ocr) not found
  running build_ext
  building 'tesseract_ocr' extension
  creating build
  creating build/temp.linux-x86_64-3.5
  gcc -pthread -Wsign-compare -DNDEBUG -g -fwrapv -O3 -Wall -Wstrict-prototypes -fPIC -I/home/it/anaconda3/include/python3.5m -c tesseract_ocr.cpp -o build/temp.linux-x86_64-3.5/tesseract_ocr.o
  cc1plus: warning: command line option ‘-Wstrict-prototypes’

In [12]:
 !pip install opencv

  Retrying (Retry(total=4, connect=None, read=None, redirect=None)) after connection broken by 'NewConnectionError('<pip._vendor.requests.packages.urllib3.connection.VerifiedHTTPSConnection object at 0x7f5deb9d66d8>: Failed to establish a new connection: [Errno 101] Network is unreachable',)': /simple/opencv/
  Could not find a version that satisfies the requirement opencv (from versions: )
No matching distribution found for opencv


In [ ]:
!pip install opencv-python

    96% |██████████████████████████████▉ | 14.9MB 50kB/s eta 0:00:11

In [9]:
!pip install --upgrade pip

    100% |████████████████████████████████| 1.3MB 327kB/s 
  Found existing installation: pip 8.1.2
    Uninstalling pip-8.1.2:
      Successfully uninstalled pip-8.1.2


In [11]:
 !pip install cv2

  Could not find a version that satisfies the requirement cv2 (from versions: )
No matching distribution found for cv2


In [21]:
!pip install Pillow

In [3]:
import math

def img_round(x, base=75):
    """
    Now useless function (replaced by binarization) for flattening image data
    """
    return (base * math.floor(float(x)/base))
vround = np.vectorize(img_round) 

def get_img(i, size):
    """
    Returns a binary image from my file directory with index i
    """
    img = Image.open('/home/ravi/OCR files/train/'+ str(i+1) + '.Bmp')
    img = img.convert("L")
    img = img.resize((size,size))
    image = np.asarray(img)
    image.setflags(write=True)
    thresh = threshold_otsu(image)
    binary = image > thresh
    return binary

def nudge_dataset(X, Y, size):
    """
    This produces a dataset 5 times bigger than the original one,
    by moving the (size x size) images around by 1px to left, right, down, up
    """
    direction_vectors = [
        [[0, 1, 0],
         [0, 0, 0],
         [0, 0, 0]],

        [[0, 0, 0],
         [1, 0, 0],
         [0, 0, 0]],

        [[0, 0, 0],
         [0, 0, 1],
         [0, 0, 0]],

        [[0, 0, 0],
         [0, 0, 0],
         [0, 1, 0]]]

    shift = lambda x, w: convolve(x.reshape((size, size)), mode='constant',
                                  weights=w).ravel()
    X = np.concatenate([X] +
                       [np.apply_along_axis(shift, 1, X, vector)
                        for vector in direction_vectors])
    Y = np.concatenate([Y for _ in range(5)], axis=0)
    return X, Y

def show_img(img):
    width = 5.0
    height = img.shape[0]*width/img.shape[1]
    f = plt.figure(figsize=(width, height))
    plt.imshow(img)

In [4]:


df = pd.read_csv('/home/ravi/OCR files/trainLabels.csv', header=0)
raw_y = np.asarray(df['Class'])
raw_x = np.asarray([get_img(i, 50) for i in df.index]).astype(float)
x = np.asarray([i.ravel() for i in raw_x])
y = raw_y
print (x.shape, y.shape)



(6283, 2500) (6283,)


In [6]:
knn = KNeighborsClassifier(n_neighbors=5)

# fit
knn.fit(X_train, Y_train)
print('RJ')

RJ


In [5]:
from sklearn.cross_validation import train_test_split
X_train, X_test, Y_train, Y_test = train_test_split(x, y)

#nudge dataset improves performance: test and see!
X_train, Y_train = nudge_dataset(X_train, Y_train, 50)


In [7]:
def get_test_img(i):
    """
    Returns image from my file directory with corresponding index i
    """
    img = Image.open('/home/ravi/OCR files/test/'+ str(i) + '.Bmp')
    img = img.convert("L")
    img = img.resize((50,50))
    image = np.asarray(img)
    image.setflags(write=True)
    thresh = threshold_otsu(image)
    binary = image > thresh
    return binary

In [9]:
from random import randint

random_image = 7062
print(random_image)
print("My Guess for this file is:" + str(knn.predict(get_test_img(random_image).ravel())).strip('[]\''))
#show_img(get_test_img(random_image))

7062
My Guess for this file is:T


/home/ravi/anaconda3/lib/python3.5/site-packages/sklearn/utils/validation.py:386: DeprecationWarning: Passing 1d arrays as data is deprecated in 0.17 and willraise ValueError in 0.19. Reshape your data either using X.reshape(-1, 1) if your data has a single feature or X.reshape(1, -1) if it contains a single sample.
  DeprecationWarning)


In [10]:

!pip3 install pytesseract

Cleaning up...


In [11]:
!pip3 install gTTS

Downloading/unpacking gTTS
  Running setup.py (path:/tmp/pip_build_ravi/gTTS/setup.py) egg_info for package gTTS
    
Downloading/unpacking gtts-token (from gTTS)
  Running setup.py (path:/tmp/pip_build_ravi/gtts-token/setup.py) egg_info for package gtts-token
    
  Running setup.py install for gTTS
    changing mode of build/scripts-3.4/gtts-cli.py from 664 to 775
    error: could not create '/usr/local/lib/python3.4/dist-packages/gtts': Permission denied
    Complete output from command /usr/bin/python3 -c "import setuptools, tokenize;__file__='/tmp/pip_build_ravi/gTTS/setup.py';exec(compile(getattr(tokenize, 'open', open)(__file__).read().replace('\r\n', '\n'), __file__, 'exec'))" install --record /tmp/pip-8g9v7mf9-record/install-record.txt --single-version-externally-managed --compile:
    running install

running build

running build_py

creating build

creating build/lib

creating build/lib/gtts

copying gtts/version.py -> build/lib/gtts

copying gtts/__init__.py -> build/lib/gt

In [20]:
import pytesseract
from gtts import gTTS
import os

from PIL import Image, ImageEnhance, ImageFilter

im = Image.open("/home/ravi/dbk.png") # the second one 
im = im.convert('RGB')
im = im.filter(ImageFilter.MedianFilter())
enhancer = ImageEnhance.Contrast(im)
im = enhancer.enhance(2)
im = im.convert('1')
im.save('/home/ravi/temp2.jpg')
text1 = pytesseract.image_to_string(Image.open('/home/ravi/dbk.png'))
text_file = open("/home/ravi/Output.txt", "w")
text_file.write(text1)
text_file.close()
tts = gTTS(text=text1, lang='en')
tts.save("/home/ravi/out.mp3")
os.system("mpg321 /home/ravi/out.mp3")
print(text1)

Dr D B Kulkarni , Professor and Head of Information Technology department, has been awarded
Intel’s micro-grant award of US$1000 for development of thread safe data structure in the area of
high performance computing.
